# title of the project 
## a list of author(s) names,


## Abstract
a few short sentences highlighting the main points of the project

## Introduction
(3-4 paragraphs) reviewing your topic, related technical ideas/algorithms, your selected methodology/approach, its motivation, and outlining the overall structure/organization of the report. 


## Contributions

## Methods
### 1. Estimate the radiometric response function from the aligned images

Estimate irradiance values $E_i$ and the radiometric response function $f$ at the same time. 

$$z_{ij} = f(E_it_j)$$ where $t_j$ is the exposure time for the $j$th image. 

The inverse response curve $f^{−1}$ is given by
$$f^{−1}(z_{ij}) = E_i t_j$$

Taking logarithms of both sides 
$$g(z_{ij})=\log f^{-1}(z_{ij})=\log E_i +\log t_j$$
 ($g$ maps pixel values $z_{ij}$ into log irradiance)
 


Also, we need make the reponse curve smooth by adding a second-order smoothness constraint
$$\lambda\sum_k g''(k)^2 = \lambda\sum[g(k-1) - 2g(k)+ g(k+1)]^2$$

Since pixel values are more reliable in the middile of their range, they also add a weight function
$$ w(z)=\left\{
\begin{aligned}
z-z_{min} & &  z \le (z_{min}+z_{max})/2 \\
z_{max}-z & & z \gt (z_{min}+z_{max})/2 \\
\end{aligned}
\right.
$$

Put all together we get a least squares problem to estimate the radiometric response function $g$ and irradiance values $E_i$

$$E = \sum_i\sum_jw(z_{i,j})[g(z_{i,j}) - \log E_i - \log t_j]^2 + \lambda\sum_k g''(k)^2$$

In other word, we are solve the two equations 
$$ w(z_{i,j}) g(z_{i,j}) - w(z_{i,j}) \log E_i  = w(z_{i,j}) \log t_j $$ 
$$\lambda[g(z_{i,j}-1) - 2g(z_{i,j})+ g(z_{i,j}+1)] = 0$$
together


#### Assume that $t_j$ is known

The response value $g_k = g(k)$, where g can be discretized according to the 256 pixel values commonly observed in eight-bit images. (The response curves are calibrated separately for each color channel.)

In [4]:
import numpy as np

In [1]:
'''
Modified from gsolve.m matlab version

Solve for imaging system response function

Given a set of pixel values observed for several pixels in several
images with different exposure times, this function returns the
imaging system’s response function


Z(i,j): the pixel values of pixel location number i in image j

B(j): the log delta t, or log shutter speed, for image j

l: lamdba, the constant that determines the amount of smoothness

w(z): the weighting function value for pixel value z

'''
zmin = 0.
zmax = 255.

def weight_hat(z):
    return min(z-zmin, zmax-z)

def gsolve(Z, B, lmd, w=weight_hat):

    locations = Z.shape[0]
    sequences = Z.shape[1]
    n = 256  # [0, 255]
    A = np.zeros((locations * sequences + n - 1, locations + n), dtype=float)
    b = np.zeros(A.shape[0], dtype=float)

    #  Include the data−fitting equations
    k = 0
    for i in range(locations):
        for j in range(sequences):
            wij = w(Z[i, j])
            A[k, int(Z[i, j])] = wij
            A[k, n + i] = -wij
            b[k] = wij * B[j]
            k += 1

    # Fix the curve by setting its middle value to 0, i.e. g(128) = 0    
    A[k, 128] = 1   
    k += 1

    # Include the smoothness equations
    for i in range(n-2):
        wi = w(i)
        A[k, i] = lmd * wi
        A[k, i+1] = -2 * lmd * wi
        A[k, i+2] = lmd * wi
        k += 1

    # Solve the system
    x = np.linalg.lstsq(A, b)[0]
    g = x[:n]
    lnE = x[n:]

    return (g, lnE)

(Note: we need to call $gsolve$ 3 times for R,G,B)

#### Sample

Question: how do we sample the pixel value $z_{ij}$? What is the sample ratio?

For $N$ sample pixels in each image and $P$ images, we need $N \times P > (Z_{max} -Z_{min}) + N$ (i.e. number of given parameters is greater than number of unknowns)

Suppose we have 3 images, $2N > 255$, $N > 128 $ should be sufficient.

- random select k pixels for each pixel value ([0,255])?

- sample based per grid? (i.e. img[::k, ::k, 0])?



In [ ]:
'''
Some Sample function here
'''

### Print the log response function and result image

Once we have the sample ($z$) and reponse function $g$, we could use it to print figure like 10.13

If the images are noise free, we can use any non-saturated pixel value to estimate the corresponding radiance by mapping it through the inverse response curve $E = g(z)$.

Debevec and Malik (1997) use a hat function (10.7) which accentuates mid-tone pixels while avoiding saturated values.

### 2. Estimate a radiance map by selecting of blending pixels from different exposures

(Copy from the book directlt lol, need to be reformated later)

Unfortunately, pixels are noisy, especially under low-light conditions when fewer photons arrive at the sensor. 

Mitsunaga and Nayar (1999) show that in order to maximize the signal-to-noise ratio (SNR), the weighting function must emphasize both higher pixel values and larger gradients in the transfer function
$$
w(z)=g(z)/g'(z)
$$


the weights $w$ are used to form the final irradiance estimate

$$
\log E_i = \frac{\sum_j w(z_{ij})[g(z_{ij})-\log t_j]}{\sum_j w(z_{ij})}
$$

### 3. Tone map the resulting high dynamic range (HDR) image back into a displayable gamut

It is usually necessary to display the HDR image on a lower gamut screen.

1. Global Transfer Curve (i.e Gamma Curve) (Larson, Rushmeier, and Pattanaik 2005)
    - If Gamma curve is applied seperate to each channel, then the color is less saturated 
    - If Gamma curve is applied to the luminance channel, then result is better. (the image is splited up into luminance and chrominance components $L*a*b$
2. If the image has wide range of exposures, we can divide each pixel by the average brightness in a region around it, like dodging and burning.

asasd

## Extentions
-  multi-exposure images taken without a tripod using homography-based registration of the sequence
    - 
- drop the assumption that exposure is known

(both are metioned in the book)

## Conclusions 
(2-4 paragraphs) summarizing your observations, results, etc.

1. Take a series of bracketed images with camera on a tripod (AEB mode)
2. Global alignment
3. Estimate radiometric response function
4. Estimate HDR radiance imgae by selecting / blending pixels from different exposure